## Proof of Concept
The output is the df that gets generated, to be then made into a csv. I've set it up to 10... but it can be set to something larger. 

In [7]:
import os
import requests
import json
import pandas as pd
import random

def calculate_price(drink):
    premium_ingredients = ["Absinthe", "Grand Marnier", "Cointreau", "Chambord", "Campari",
                            "Green Chartreuse", "Añejo Rum", "Midori Melon Liqueur", "Frangelico",
                            "Galliano", "St. Germain Elderflower Liqueur", "Bailey's Irish Cream",
                            "Kahlua", "Amaretto", "Drambuie", "Benedictine", "Dom Pérignon",
                            "Patrón Tequila", "Château Margaux", "Hennessy Cognac"]

    base_price = 6.5 

    # Additional pricing based on factors
    price_per_ingredient = 0.75
    premium_charge = 2.5
    alcoholic_price_increase = 3 

    price = base_price + drink['num_ingredients'] * price_per_ingredient

    # surcharge for specific spirits
    for ingredient in drink['ingredients'].split(', '):
        if ingredient in premium_ingredients:
            price += premium_charge
            # Add a random factor for each premium ingredient
            price += random.uniform(-0.25, 1)

    # Add additional price for alcoholic drinks
    if drink['alcoholic']:
        price += alcoholic_price_increase

    # Add a general random factor
    price += random.uniform(-1, 1)

    return round(price, 2)


def generate_cocktail_dataframe(num_cocktails):
    cocktail_id = [] 
    names = []
    ingredients = [] 
    num_ingredients = []  
    alcoholic = []
    acidic = []
    caffiene = [] 
    category = []  
    glass = []  
    sugary = []
    instructions = []  
    thumb_urls = []    

    sweet_list = [
    'Grenadine', 'Pineapple juice', 'Amaretto', 'Sour mix',
    'Kahlua', 'Powdered sugar', 'Banana Liqueur', 'Chocolate liqueur', 
    'Peach schnapps', 'Orange Juice', 'Soda water', 'Vanilla syrup',
    'Baileys irish cream', 'White Creme de Menthe', 'Sprite'
    'Creme de Cacao','Surge', 'Honey', 'Cocoa powder', ' 7-up',
    'Brandy', 'Cranberry juice', 'Ginger ale', 'Peach Vodka', 'Coca-Cola', 'Coke',
    'Creme de Cassis', 'Champagne', 'Angostura bitters', 'Tonic water',
    'Grapefruit Juice', 'Agave Syrup', 'Soda Water', 'Zima',
    'Chambord raspberry liqueur', 'Vodka', 'Sugar', 'Sugar Syrup',
    'Passion fruit juice', 'Prosecco', 'Elderflower cordial', 'Jägermeister',
    'Root beer', 'Corn syrup', 'syrup',
    'Butterscotch schnapps', 'Vanilla Ice-Cream',
    'Sherry', 'Grand Marnier', 'Orange juice', 'Lemonade', 'Orange spiral',
    'Sweet Vermouth', 'Sloe gin', 'Apricot brandy', 'Raspberry syrup', 'Dry Vermouth'
    ]


    url = "https://www.thecocktaildb.com/api/json/v1/1/random.php"

    # Generate random cocktails that are valid
    while len(cocktail_id) < num_cocktails:
        r = requests.get(url)

        if r.status_code == 200:
            site_json = r.json()

            # Check if the response contains valid data
            if 'drinks' in site_json and site_json['drinks']:
                drink = site_json['drinks'][0]
                name = drink.get('strDrink')

                # Append the columns 
                cocktail_id.append(drink['idDrink'])
                names.append(name)

                # Extract ingredients (removing Nones)
                ingredients_raw = [drink.get(f'strIngredient{i}', '') for i in range(1, 16)]
                ingredients.append(', '.join(filter(None, ingredients_raw)))

                # Calculate and append the number of ingredients
                num_ingredients.append(len(list(filter(None, ingredients_raw))))

                # Convert "strAlcoholic" to boolean
                alcoholic_status = drink.get('strAlcoholic', '').lower() == 'alcoholic'
                alcoholic.append(alcoholic_status)

                # Check if any of the specified ingredients are in the list
                acidic_status = any(ingredient in ["Lemon", "Lime", "Orange"] for ingredient in ingredients_raw)
                acidic.append(acidic_status)

                caf_status = any(ingredient in ["Coffee", "Espresso"] for ingredient in ingredients_raw) 
                caffiene.append(caf_status)

                category.append(drink.get('strCategory', ''))
                glass.append(drink.get('strGlass', ''))

                sugar_score = 0
                sugar_score = sum(1 for ingredient in ingredients_raw if ingredient in sweet_list)
                sugary.append(sugar_score)

                instructions.append(drink.get('strInstructions', ''))
                thumb_urls.append(drink.get('strDrinkThumb', ''))


            else:
                continue

    # Create DataFrame
    df_cocktail = pd.DataFrame({
        "cocktail_id": cocktail_id,
        "cocktail_name": names,
        "num_ingredients": num_ingredients,
        "ingredients": ingredients,
        "alcoholic": alcoholic,
        "sugary_score":sugary,
        "acidic": acidic,
        "caffiene": caffiene, 
        "category": category, 
        "glass": glass,
        "instructions":instructions,
        "image_url": thumb_urls
    })  

    df_cocktail["price"] =  df_cocktail.apply(calculate_price, axis=1)

    # Drop duplicates based on 'cocktail_id'
    df_cocktail = df_cocktail.drop_duplicates(subset=['cocktail_id'])   

    csv_filename = "cocktails_result.csv"
    csv_path = os.path.join(os.getcwd(), csv_filename)
    df_cocktail.to_csv(csv_path, index=False)
    print(f"Data saved to {csv_path}")

    pd.set_option('display.max_colwidth', None) 

    return df_cocktail


generate_cocktail_dataframe(10)


Data saved to c:\Users\andre\Desktop\Personal\CocktailDB_API\cocktails_result.csv


,cocktail_id,cocktail_name,num_ingredients,ingredients,alcoholic,sugary_score,acidic,caffiene,category,glass,instructions,image_url,price
0,11013,Alaska Cocktail,4,"Orange bitters, Gin, Yellow Chartreuse, Lemon peel",True,0,False,False,Ordinary Drink,Cocktail glass,"Stir all ingredients with ice, strain contents into a cocktail glass. Drop in a twist of lemon and serve.",https://www.thecocktaildb.com/images/media/drink/wsyryt1483387720.jpg,12.33
1,17840,Affinity,4,"Scotch, Sweet Vermouth, Dry Vermouth, Orange bitters",True,2,False,False,Ordinary Drink,Cocktail glass,"In a mixing glass half-filled with ice cubes, combine all of the ingredients. Stir well. Strain into a cocktail glass.",https://www.thecocktaildb.com/images/media/drink/wzdtnn1582477684.jpg,12.51
2,11239,Cherry Rum,3,"Light rum, Cherry brandy, Light cream",True,0,False,False,Ordinary Drink,Cocktail glass,"Shake all ingredients with ice, strain into a cocktail glass, and serve.",https://www.thecocktaildb.com/images/media/drink/twsuvr1441554424.jpg,11.97
3,17249,Pegu Club,5,"Gin, Orange Curacao, Lime Juice, Angostura Bitters, Orange Bitters",True,0,False,False,Cocktail,Cocktail glass,"Shake, strain, up, cocktail glass",https://www.thecocktaildb.com/images/media/drink/jfkemm1513703902.jpg,14.21
4,11020,Algonquin,3,"Blended whiskey, Dry Vermouth, Pineapple juice",True,2,False,False,Ordinary Drink,Cocktail glass,"Combine and shake all ingredients with ice, strain contents into a cocktail glass, and serve.",https://www.thecocktaildb.com/images/media/drink/uwryxx1483387873.jpg,11.32
5,11634,Lady Love Fizz,6,"Gin, Light cream, Powdered sugar, Lemon, Egg white, Carbonated water",True,1,True,False,Ordinary Drink,Cocktail glass,"Shake all ingredients (except carbonated water) with ice and strain into a cocktail glass over two ice cubes. Fill with carbonated water, stir, and serve.",https://www.thecocktaildb.com/images/media/drink/20d63k1504885263.jpg,13.19
6,12696,Lassi - Sweet,5,"Yoghurt, Water, Sugar, Salt, Lemon juice",False,1,False,False,Other / Unknown,Highball glass,Put all ingredients into a blender and blend until nice and frothy. Serve chilled.,https://www.thecocktaildb.com/images/media/drink/9jeifz1487603885.jpg,10.63
7,11326,Dubonnet Cocktail,4,"Dubonnet Rouge, Gin, Bitters, Lemon peel",True,0,False,False,Ordinary Drink,Cocktail glass,Stir all ingredients (except lemon peel) with ice and strain into a cocktail glass. Add the twist of lemon peel and serve.,https://www.thecocktaildb.com/images/media/drink/pfz3hz1582483111.jpg,13.29
8,12560,Afterglow,3,"Grenadine, Orange juice, Pineapple juice",False,3,False,False,Cocktail,Highball Glass,Mix. Serve over ice.,https://www.thecocktaildb.com/images/media/drink/vuquyv1468876052.jpg,7.97
9,15082,Royal Flush,4,"Crown Royal, Peach schnapps, Chambord raspberry liqueur, Cranberry juice",True,3,False,False,Shot,Old-fashioned glass,Pour all the ingredients into tumbler over ice. Strain into glass.,https://www.thecocktaildb.com/images/media/drink/7rnm8u1504888527.jpg,12.39


In [8]:
df_cocktails = generate_cocktail_dataframe(500)
df_cocktails

Data saved to c:\Users\andre\Desktop\Personal\CocktailDB_API\cocktails_result.csv


,cocktail_id,cocktail_name,num_ingredients,ingredients,alcoholic,sugary_score,acidic,caffiene,category,glass,instructions,image_url,price
0,14482,Darkwood Sling,4,"Cherry Heering, Soda water, Orange juice, Ice",True,2,False,False,Soft Drink,Cocktail glass,"There are many good cherry liqueurs you can use, but I prefere Heering. Add one share of the liqueur. Then you add one share of Soda. For a sour sling use Tonic (most people prefer the drink without Tonic). Afterwards you fill the glass with Orange Juice and ice cubes.",https://www.thecocktaildb.com/images/media/drink/sxxsyq1472719303.jpg,13.32
1,178340,Garibaldi Negroni,4,"Gin, Campari, Orange Juice, Orange Peel",True,1,False,False,Cocktail,Highball glass,Mix together in a shaker and garnish with a simple orange slice. Fill your vitamin C and cocktail fix at the same time!,https://www.thecocktaildb.com/images/media/drink/kb4bjg1604179771.jpg,16.52
2,178350,The Galah,5,"Dark Rum, Campari, Creme De Banane, Pineapple Juice, Lime Juice",True,0,False,False,Cocktail,Collins glass,Mix together the alcoholic portions and top with Pineapple and Lime juice.,https://www.thecocktaildb.com/images/media/drink/sy7y6r1614775067.jpg,15.62
3,17246,Empellón Cocina's Fat-Washed Mezcal,3,"Mezcal, Chocolate liqueur, Coffee liqueur",True,1,False,False,Cocktail,Beer Glass,"To ensure that your pork fat is just as delicious as theirs, here’s their adobo marinade and what to do with it (you’ll also need a rack of ribs):\r\n\r\n4 ancho chiles, 8 guajillo chiles and 4 chipotle chiles, plus 4 cloves roasted garlic, half a cup of cider vinegar, a quarter teaspoon of Mexican oregano, 1 teaspoon of ground black pepper, a whole clove, a quarter teaspoon of ground cinnamon and a half teaspoon of ground cumin.\r\n\r\nToast the dried chiles and soak in water for at least an hour until they are rehydrated. Drain and discard the soaking liquid. Combine the soaked chiles with the remaining ingredients and purée until smooth.\r\n\r\nCold smoke a rack of baby back pork ribs by taking a large hotel pan with woodchips on one side and charcoal on the other. Place another, smaller, pan with pork ribs, above the charcoal/woodchip pan. Ignite the charcoal, being careful to not ignite the woodchips. Cover both pans with foil and allow to smoke for 10-15 minutes, until desired level of smoke is achieved, then coat with adobo marinade and wrap in tin foil prior to placing ribs in a 300 degree oven for 7 hours. When the ribs have cooled, strain off the fat and use for the infusion.\r\n\r\nIf you’re having a hard time coming up to the same kind of volume of fat, make up the balance with pork lard from a butcher. To get the same depth of flavor without the ribs, heat up the fat in a pot with a few spoons of the marinade.\r\n\r\nOnce you’ve got your tub of seasoned pork fat in cooled liquid form, pour equal amounts of Ilegal Joven mezcal and fat into a sealable container. Seal the container and give it a really good shake, then put it in the freezer overnight. When the whole thing is separated and congealed, pour it through a fine mesh chinoise. If you don’t have a chinoise, try a fine mesh strainer, or if you don’t have one of those, try spooning off most of the fat. There will be some beads of orange fat left in the strained mezcal: run that through a few layers of cheesecloth (or coffee filters in a pinch) to get rid of the last of it.\r\n\r\nThe mezcal is now ready for drinking, straight-up or in a cocktail. \r\n\r\nHabanero tincture\r\n\r\nSlice habaneros and add 2 ounces Ilegal Joven mezcal.\r\nAllow to sit overnight or until desired level of heat is achieved.\r\nCocktail\r\n\r\nCombine mezcal and chocolate liqueur in a mixing glass with ice and stir for 45 seconds.\r\nStrain into chilled coupe.\r\nCarefully ""sink"" the coffee liqueur down the inside of the coupe over a spoon.\r\nGarnish with 5 drops habanero tincture.",https://www.thecocktaildb.com/images/media/drink/osgvxt1513595509.jpg,11.59
4,17177,Dark Caipirinha,3,

In [49]:
import pandas as pd

# Recommendation System 
def drink_rec(num_ingred: tuple, alc, spec_ingred, sugar, acid, caf, price):
    df_cocktails = pd.read_csv('cocktails_result_346.csv')

    df_cocktails['ingredients_lower'] = df_cocktails['ingredients'].str.lower()

    df_rec = df_cocktails[
        (df_cocktails['num_ingredients'].between(num_ingred[0], num_ingred[1])) &
        (df_cocktails['alcoholic'] == alc) & 
        (df_cocktails['sugary_score'] == sugar) & 
        (df_cocktails['acidic'] == acid) &
        (df_cocktails['caffiene'] == caf) & 
        (df_cocktails['price'] < price) &
        (df_cocktails['ingredients_lower'].str.contains(spec_ingred))
    ] 

    df_rec.drop(columns=['ingredients_lower']) 

    # TEMPORARILY setting the column width size to None... will revert after function call
    pd.set_option('display.max_colwidth', None)

    return df_rec[['cocktail_name', 'num_ingredients', 'ingredients', 'alcoholic', 'sugary_score', 'acidic', 'caffiene', 'glass', 'instructions', 'image_url', 'price']]

pd.reset_option("display.max_colwidth")


# Improvements -- change sugar to a tuple value as well (same as num_ingred)
drink_rec((2,15), False, "water", 1, False, True, 14)



,cocktail_name,num_ingredients,ingredients,alcoholic,sugary_score,acidic,caffiene,glass,instructions,image_url,price
119,Iced Coffee,4,"Coffee, Sugar, Water, Milk",False,1,False,True,Coffee mug,Mix together until coffee and sugar is dissolved. Add milk. Shake well. Using a blender or milk shake maker produces a very foamy drink. Serve in coffee mug.,https://www.thecocktaildb.com/images/media/drink/ytprxy1454513855.jpg,8.63


In [44]:
import altair as alt

alt.Chart(df_cocktails).mark_point(filled=True, opacity=0.6).encode(
    x='num_ingredients',
    y='price',
    color='acidic'
    #  size=alt.Size('price:Q', title='Price', scale=alt.Scale(range=[0.01, 300]))
)

alt.Chart(...)